Interesting LDA article: https://medium.com/data-science/practical-guide-to-topic-modeling-with-lda-05cd6b027bdf

In [2]:
!pip install gensim

In [3]:
%pip install pyLDAvis

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 29 13:54:48 2025

@author: dnd2129
NLP processes
"""

import pandas as pd
#%pip install "numpy<2"
import numpy as np
#%pip install spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import punkt
from nltk import word_tokenize, ngrams
from nltk.util import ngrams
#%pip install spacy
import spacy
#spacy.cli.download("en_core_web_sm")
import gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools

import pyLDAvis
#import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
df = pd.read_excel("Student Course Elective Enrollments Graduates 2016-2025.xlsx")
df.head()
#unq_courses= df.combined_name.unique()



,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined Native Flag,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities


#NLP Process

## Cleaning Text

In [6]:

#cleaning texts
df['clean_name'] = df['combined_name'].fillna('') #changing nan to blank
df['clean_name'] = df['clean_name'].str.replace(r'\s+', ' ', regex=True)#cleaning new line chars
df['clean_name'] = df['clean_name'].str.replace("'", '', regex=False)#removing single quotes

df.head()


,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities


### Remove stopwords

In [7]:
def rem_sw(var_in):
    sw = list(set(stopwords.words('english')))
    tmp = [word for word in var_in.split() if word not in sw]
    return ' '.join(tmp)

In [8]:
df['clean_name'] = df['clean_name'].apply(rem_sw)
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities


In [34]:
df.tail(20)

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
147028,147028,C004355340,bcr2132,2023,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Investment Portfolio Management,Financial Services,NaN,
147029,147029,C004356182,ljr2158,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,NaN,
147030,147030,C004360272,cnf2112,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP+,...,No Race Reported,No Race Reported,No Race Reported,NaN,NaN,United States,NaN,NaN,NaN,
147031,147031,C004360287,aaa2371,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP+,...,International,International,International,Spain,France,Spain,Venture Capital/Private Equity,Financial Services,NaN,
147032,147032,C004361262,kj2525,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP+,...,International,International,International,India,NaN,India,Venture Capital/Private Equity,Financial Services,NaN,
147033,147033,C004410818,mad2313,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Retail / Wholesale / Luxury Goods,NaN,
147034,147034,C004436778,rgp2130,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP+,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Brand/Product Management,Retail / Wholesale / Luxury Goods,NaN,
147035,147035,C004437064,jg4334,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,H,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Business Development/ Product Development,Education,NaN,
147036,147036,C004437306,jh4363,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Consulting,Education,NaN,
147037,147037,C004437766,rlm2216,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,H,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Social Enterprise/ Not-For-Profit,NaN,


## n-gram extraction

### Manual Tokenizing::: may be ignore as tfidf may be better!

In [ ]:
'''
Tokenize each sentence into a list of words, removing punctuations and
unnecessary characters altogether.

Using spaCy because nltk wasn't working-- and according to article, spaCy is better anyway....
'''
nlp = spacy.load("en_core_web_sm")

def generate_ngrams_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]

    unigrams = list(zip(tokens))
    bigrams = list(zip(tokens, tokens[1:])) if len(tokens) > 1 else []
    trigrams = list(zip(tokens, tokens[1:], tokens[2:])) if len(tokens) > 2 else []

    return unigrams, bigrams, trigrams


In [ ]:
df[['unigrams', 'bigrams', 'trigrams']] = df['clean_name'].fillna('').apply(
    lambda x: pd.Series(generate_ngrams_spacy(x))
)

In [ ]:
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,token_name,unigrams,bigrams,trigrams
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1315...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1315...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1502...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1502...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1503...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"


In [ ]:
#df.to_csv('course names uni-bi-trigrams.csv')

In [ ]:
subset = df[['clean_name', 'unigrams', 'bigrams', 'trigrams']].copy()

# Convert unhashable list columns to strings
subset['unigrams'] = subset['unigrams'].astype(str)
subset['bigrams'] = subset['bigrams'].astype(str)
subset['trigrams'] = subset['trigrams'].astype(str)

# Now drop duplicates safely
unique_subset = subset.drop_duplicates()

# Check it
unique_subset.head()


,clean_name,unigrams,bigrams,trigrams
0,Winning Strategic Capabilities,"[('winning',), ('strategic',), ('capabilities',)]","[('winning', 'strategic'), ('strategic', 'capa...","[('winning', 'strategic', 'capabilities')]"
266,Web App Programming Python,"[('web',), ('app',), ('programming',), ('pytho...","[('web', 'app'), ('app', 'programming'), ('pro...","[('web', 'app', 'programming'), ('app', 'progr..."
581,Virtual Reality & The Future Storytelling: How...,"[('virtual',), ('reality',), ('the',), ('futur...","[('virtual', 'reality'), ('reality', 'the'), (...","[('virtual', 'reality', 'the'), ('reality', 't..."
742,Virtual Reality & Artificial Intelligence: How...,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."
817,Virtual Reality & Artificial Intelligence,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."


In [ ]:
#unique_subset.to_csv("unique_subset.csv", index=False)

Manually looking at unigrams, bigrams and trigrams to decide which to use for LDA

### Tf-idf

In [ ]:
# Your existing setup
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df['clean_name'])

# Sum TF-IDF values across all documents
tfidf_sums = X.sum(axis=0).A1  # A1 flattens the matrix to a 1D array
feature_names = vectorizer.get_feature_names_out()

gives feature names, i.e., ngrams

getting  SCORE FOR EACH ngram

In [ ]:
# Create DataFrame of words and scores
tfidf_scores = pd.DataFrame({
    'term': feature_names,
    'score': tfidf_sums
})

# Sort by score descending
tfidf_scores = tfidf_scores.sort_values(by='score', ascending=False)

# View top words/phrases
print(tfidf_scores.head(30))

                             term        score
1870                   managerial  3291.995459
1874      managerial negotiations  3210.213805
2093                 negotiations  3210.213805
1097                      finance  3071.540406
2698                     strategy  3069.928453
1946                      markets  3036.397317
124                      analysis  2875.461116
1621                    investing  2742.828802
418                       capital  2625.593284
1667                  investments  2595.576559
424               capital markets  2589.281199
1953          markets investments  2580.466807
425   capital markets investments  2580.466807
468                        change  2152.975640
2790                          the  2074.443045
350                      business  2068.745496
1843                   management  2016.757233
2160               organizational  1961.051792
1163                  foundations  1958.536477
2161        organizational change  1953.261068
2980         

write csv

In [ ]:
tfidf_scores.to_csv("tfidf_scores ngrams 1,2,3.csv", index=False)

### IDF: maybe this will make it easier to sort out words!
higher scores--> less frequent-- may be more valuable to us

In [24]:
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Use CountVectorizer to get n-grams (1 to 3)
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='word')
X = vectorizer.fit_transform(df['clean_name'])

# Step 2: Extract n-gram feature names
feature_names = vectorizer.get_feature_names_out()

# Step 3: Get number of documents each n-gram appears in (document frequency)
# This is a binary presence matrix — whether term appears in document
document_frequencies = (X > 0).sum(axis=0).A1  # .A1 flattens to 1D array
num_documents = X.shape[0]


In [30]:
print(feature_names)

['101' '101 driving' '101 driving strategic' ... 'your startup'
 'your startup ii' 'your startup tech']


In [25]:
# Step 4: Compute smoothed IDF scores
idf_scores = {
    term: math.log((1 + num_documents) / (1 + dfreq)) + 1
    for term, dfreq in zip(feature_names, document_frequencies)
}

In [26]:
# Step 5: Convert to DataFrame and sort
idf_df = pd.DataFrame(idf_scores.items(), columns=["term", "idf_score"])
idf_df['df_count'] = document_frequencies #adding number of documents(coursename) each term appears in
#matches substring and \b ..\b matches word boundaries... so 'database' won't count in 'data'.

idf_df = idf_df.sort_values(by="idf_score", ascending=False)

idf_df.head(20)



,term,idf_score,df_count
0,101,12.205374,1
1,101 driving,12.205374,1
1370,high tech firms,12.205374,1
1418,immersion exploring turkeys,12.205374,1
1495,in financial,12.205374,1
1496,in financial markets,12.205374,1
1517,industries integrating,12.205374,1
1518,industries integrating mba,12.205374,1
1522,inference,12.205374,1
1529,information in,12.205374,1


Add count of unique course_identifiers which have each term

In [35]:
idf_df['term_frequencies_in_unique_coursenames'] = idf_df['term'].apply(
    lambda term: df['clean_name']
        .str.lower()
        .str.replace(r'[^a-z0-9 ]+', '', regex=True)
        .str.contains(re.escape(term.lower()), case=False)
        .sum()
)
idf_df.head(20)


,term,idf_score,df_count,term_frequencies_in_unique_coursenames
0,101,12.205374,1,1
1,101 driving,12.205374,1,1
1370,high tech firms,12.205374,1,1
1418,immersion exploring turkeys,12.205374,1,1
1495,in financial,12.205374,1,1
1496,in financial markets,12.205374,1,1
1517,industries integrating,12.205374,1,1
1518,industries integrating mba,12.205374,1,1
1522,inference,12.205374,1,1
1529,information in,12.205374,1,2


In [36]:
idf_df.tail(10)

,term,idf_score,df_count,term_frequencies_in_unique_coursenames
1667,investments,4.151487,6291,6291
418,capital,4.079743,6759,6889
1843,management,3.993348,7369,7369
1097,finance,3.770476,9209,9209
1621,investing,3.765818,9252,9252
350,business,3.753573,9366,10346
124,analysis,3.718640,9699,9699
2698,strategy,3.696616,9915,9915
1946,markets,3.652331,10364,10364
2790,the,3.538468,11614,12482


In [ ]:
# Step 6: Export to CSV
idf_df.to_csv("idf_scores_ngrams.csv", index=False)



# LDA processes

now, combine all courses for each student into one row so we have one row per studnt. Then,tokenize it like I did above (choose uni/bi or trigrams depending of which worked best).
THEN: only retain the tokens that I manually chose to use from the manual mapping.

### Grouping by student
One row per student-- adding a course_list column which contains all courses they have taken.

In [ ]:
'''
Check the Sparsicity
'''
# Materialize the sparse data
data_dense = data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
#Sparsicity:  0.6425770811326413 %

Sparsicity:  0.6425770811326413 %


In [ ]:
#One row per student-- adding a course_list column which contains all courses they have taken.
grouped = df.groupby('uni')

## LDA

Checking an LDA model with arbitrary number of components

In [ ]:
'''
Build LDA model with sklearn
'''

lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


Checking model performance overall

In [ ]:
'''
Diagnose model performance with perplexity and log-likelihood

'''
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
#Log Likelihood:  -2141711.0363983824
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
#Perplexity:  133.36523412184394
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2140563.6769724055
Perplexity:  133.01609743189883
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### RandomSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np

# Define the parameter distributions
param_dist = {
# Init the Model
    'n_components': [1,2,3,4,5],              # Number of topics
    'learning_decay': [.7],        #according to article, learning decay SHOULD be >.5, and around .7 is optimal.
    'batch_size': [2000],
}

# Initialize the LDA model
lda = LatentDirichletAllocation(learning_method='online', random_state=42)

# Initialize Randomized Search
random_search = RandomizedSearchCV(
    estimator=lda,
    param_distributions=param_dist,
    n_iter=10,                       # Number of parameter combinations to try
    scoring=None,         # Or another relevant metric
    cv=3,
    verbose=1,
    random_state=42
)

# Fit to your vectorized data
random_search.fit(data_vectorized)

# Best model and its parameters
print("Best Model's Params:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

'''
Best Model's Params: {'n_components': 5, 'learning_decay': 0.7, 'batch_size': 2000}
Best Score: nan
'''

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best Model's Params: {'n_components': 1, 'learning_decay': 0.7, 'batch_size': 2000}
Best Score: -1100419.5543143193


"\nBest Model's Params: {'n_components': 5, 'learning_decay': 0.7, 'batch_size': 2000}\nBest Score: nan\n"

### GridSearch
for the best model for n_components (number of topics) and learning decay

In [ ]:
'''
GridSearch the best LDA model
'''
# Define Search Param
search_params = {'n_components': [1,2,5,6,10], 'learning_decay': [.7], 'batch_size': [2000]} #according to article, learning decay SHOULD be >.5, and around .7 is optimal.
# Init the Model
lda = LatentDirichletAllocation()
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, n_jobs=-1) #n_jobs=-1 -->parallelizing the process

# Do the Grid Search
model.fit(data_vectorized)

'''
Best model and Best parameters
'''
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

'''
n_componenets: 10,20,30; decay: .7,.9:
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -744749.1951048326
Model Perplexity:  96.56532804852921

'''

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 1}
Best Log Likelihood Score:  -656085.984851287
Model Perplexity:  195.08960346335755


"\nn_componenets: 10,20,30; decay: .7,.9:\nBest Model's Params:  {'learning_decay': 0.7, 'n_components': 10}\nBest Log Likelihood Score:  -744749.1951048326\nModel Perplexity:  96.56532804852921\n\n"

### Plotting Performance scores
(log_likelihood) to see best parameters

In [ ]:
'''
Compare LDA Model Performance Scores
'''

import matplotlib.pyplot as plt
results = model.best_params_

# Initialize lists
n_topics_5 = [1,2,5,10]
n_topics_7 = [1,2,5,10]
log_likelyhoods_5 = []
log_likelyhoods_7 = []

# Loop through results and collect scores
for i, params in enumerate(results['params']):
    if params['learning_decay'] == 0.5:
        log_likelyhoods_5.append(results['mean_test_score'][i])
    if params['learning_decay'] == 0.7:
        log_likelyhoods_7.append(results['mean_test_score'][i])

# Optional debug prints
print(f"Found {len(n_topics_5)} entries for learning_decay=0.5")
print(f"Found {len(n_topics_7)} entries for learning_decay=0.7")

# Plotting
plt.figure(figsize=(8, 5))
plt.xlim(0, 10)

if log_likelyhoods_5:
    combined_5 = sorted(zip(n_topics_5, log_likelyhoods_5))
    n_topics_sorted_5, log_likelyhoods_5_sorted = zip(*combined_5)
    plt.plot(n_topics_sorted_5, log_likelyhoods_5_sorted, marker='o', label='learning_decay = 0.5')

if log_likelyhoods_7:
    combined_7 = sorted(zip(n_topics_7, log_likelyhoods_7))
    n_topics_sorted_7, log_likelyhoods_7_sorted = zip(*combined_7)
    plt.plot(n_topics_sorted_7, log_likelyhoods_7_sorted, marker='s', label='learning_decay = 0.7')

plt.title("Choosing Optimal LDA Model")
plt.xlabel("Number of Topics")
plt.ylabel("Log Likelihood Score")
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'model' is not defined

In [ ]:
print("Topic numbers for decay 0.7:", sorted(set(n_topics_7)))
print("Topic numbers for decay 0.9:", sorted(set(n_topics_9)))

Topic numbers for decay 0.7: [10, 20, 30]
Topic numbers for decay 0.9: [10, 20, 30]


### Document-Topic Matrix

In [ ]:
'''
dominant topic in each document
'''
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)
def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)



In [ ]:
'''
Review topics distribution across documents
'''
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']


In [ ]:
'''
visualize the LDA model with pyLDAvis
'''
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')

In [ ]:
'''
see the Topic’s keywords
'''

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

'''
Top 15 keywords
'''
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

In [ ]:
'''
cluster documents that share similar topics and plot
'''
# Construct the k-means clusters
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=15, random_state=100).fit_predict(lda_output)
# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)
# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]
# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))
# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))


In [ ]:
'''
Plot clusters
'''
# Plot
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )